In [5]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions
from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", lambda x: '%.3f' % x)



In [6]:
# SPOILER: we will use this dataset for RFM Analysis so we load it here to use cleaned data directly.
yol = r'C:/Users/nacio/.cache/kagglehub/datasets/mashlyn/online-retail-ii-uci/versions/3/online_retail_II.csv'


df = pd.read_csv(yol, encoding='ISO-8859-1')

print("✅ df is defined")
df.head()

✅ df is defined


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [7]:
# Load the data
data_ = pd.read_csv("C:/Users/nacio/.cache/kagglehub/datasets/mashlyn/online-retail-ii-uci/versions/3/online_retail_II.csv")
data = data_.copy()

In [8]:
# EDA function
def Df_info(df):
    print("########### Head ###########")
    print(df.head())
    print("########### Tail ###########")
    print(df.tail())
    print("########### Shape ###########")
    print(df.shape)
    print("########### Info ###########")
    print(df.info())
    print("########### Info ###########")
    print(df.columns)
    print("########### Quantiles ###########")
    print(df.describe().T)
    print("########### NA ###########")
    print(df.isnull().sum())

Df_info(data)


########### Head ###########
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

           InvoiceDate  Price  Customer ID         Country  
0  2009-12-01 07:45:00  6.950    13085.000  United Kingdom  
1  2009-12-01 07:45:00  6.750    13085.000  United Kingdom  
2  2009-12-01 07:45:00  6.750    13085.000  United Kingdom  
3  2009-12-01 07:45:00  2.100    13085.000  United Kingdom  
4  2009-12-01 07:45:00  1.250    13085.000  United Kingdom  
########### Tail ###########
        Invoice StockCode                      Description  Quantity  \
1067366  581587     22899     CHILDREN'S APRON DOLLY GIRL          6   
1067367  58

In [9]:
print("The number of customer in the dataset:", data["Customer ID"].nunique()) 
#number of unique customers
print("The number of invoice in the dataset:", data["Invoice"].nunique())
#number of unique invoices
print("The sum of quantity in the dataset:", data["Quantity"].sum()) 
#total quantity of products sold
total_price_of_df = ((data["Quantity"] * data["Price"]).sum()).astype(int)   
print("The sum of price in the dataset:", total_price_of_df)
#total revenue generated
print("the sum of products in the dataset:", data["Description"].count()) 
#total number of products sold


The number of customer in the dataset: 5942
The number of invoice in the dataset: 53628
The sum of quantity in the dataset: 10608492
The sum of price in the dataset: 19287250
the sum of products in the dataset: 1062989


In [10]:
data.sort_values(by = "Price", ascending  = False).head()
#dataset have negative quantity and price values which indicates returns and refunds i will clean these values in the next steps

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
748142,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.000,15098.000,United Kingdom
241824,C512770,M,Manual,-1,2010-06-17 16:52:00,25111.090,17399.000,United Kingdom
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.090,NaN,United Kingdom
320581,C520667,BANK CHARGES,Bank Charges,-1,2010-08-27 13:42:00,18910.690,NaN,United Kingdom
1050063,C580605,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:36:00,17836.460,NaN,United Kingdom


In [11]:
data.sort_values(by = "Price", ascending  = True).head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
179403,A506401,B,Adjust bad debt,1,2010-04-29 13:36:00,-53594.360,NaN,United Kingdom
276274,A516228,B,Adjust bad debt,1,2010-07-19 11:24:00,-44031.790,NaN,United Kingdom
403472,A528059,B,Adjust bad debt,1,2010-10-20 12:04:00,-38925.870,NaN,United Kingdom
825444,A563186,B,Adjust bad debt,1,2011-08-12 14:51:00,-11062.060,NaN,United Kingdom
825445,A563187,B,Adjust bad debt,1,2011-08-12 14:52:00,-11062.060,NaN,United Kingdom


In [12]:
def Df_info2(df):
    # customer ID in dataset
    print("The number of customer in the dataset:", df["Customer ID"].nunique())
    print("-------------------------------------------------------")
    # number of unique invoices
    print("The number of invoice in the dataset:", df["Invoice"].nunique())
    print("-------------------------------------------------------")
    # total quantity of products sold
    print("The sum of quantity in the dataset:", df["Quantity"].sum())
    print("-------------------------------------------------------")
    # total revenue generated
    total_price_of_df = ((df["Quantity"] * df["Price"]).sum()).astype(int)
    print("The sum of price in the dataset:", total_price_of_df)
    print("-------------------------------------------------------")
    # most expensive product sorted by salary
    print("Our dataset, sort by highest salary")
    print(df.sort_values(by = "Price", ascending  = False).head())
    print("-------------------------------------------------------")
    # least expensive product sorted by salary
    print("Our dataset, sorted by lowest salary")
    print(df.sort_values(by = "Price", ascending  = True).head())

In [13]:
#prepearing the dataset for analysis
data.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom


In [14]:
# Changing the type of the 'InvoiceDate' column
data["InvoiceDate"] = data["InvoiceDate"].apply(pd.to_datetime)

In [15]:
# Dropping the null values in the dataset.
data.dropna(inplace = True)

In [16]:
# The state of the dataset after clearing the null values.
Df_info(data)

########### Head ###########
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  
0 2009-12-01 07:45:00  6.950    13085.000  United Kingdom  
1 2009-12-01 07:45:00  6.750    13085.000  United Kingdom  
2 2009-12-01 07:45:00  6.750    13085.000  United Kingdom  
3 2009-12-01 07:45:00  2.100    13085.000  United Kingdom  
4 2009-12-01 07:45:00  1.250    13085.000  United Kingdom  
########### Tail ###########
        Invoice StockCode                      Description  Quantity  \
1067366  581587     22899     CHILDREN'S APRON DOLLY GIRL          6   
1067367  581587  

In [ ]:
''' When I see the chapter on shape, I see deleted some values and When Check the chapter on NA, I see There are not null values. But I still have a problem, I see There is a negative value in the Quartiles chapter. I must get rid of it.

The reason why we still have negative values in the Quantity section is invoices that start with C at the beginning because these invoices are return invoices. Show to you the values:    '''

In [17]:
# Values starting with 'C'
data[data["Invoice"].str.contains("C", na = False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
178,C489449,22087,PAPER BUNTING WHITE LACE,-12,2009-12-01 10:33:00,2.950,16321.000,Australia
179,C489449,85206A,CREAM FELT EASTER EGG BASKET,-6,2009-12-01 10:33:00,1.650,16321.000,Australia
180,C489449,21895,POTTING SHED SOW 'N' GROW SET,-4,2009-12-01 10:33:00,4.250,16321.000,Australia
181,C489449,21896,POTTING SHED TWINE,-6,2009-12-01 10:33:00,2.100,16321.000,Australia
182,C489449,22083,PAPER CHAIN KIT RETRO SPOT,-12,2009-12-01 10:33:00,2.950,16321.000,Australia


In [18]:
# i have to get rid of the invoices starting with 'C'
data = data[~data["Invoice"].str.contains("C", na = False)]


In [19]:
# Removing negative and zero values from 'Quantity' and 'Price' columns
data = data[data["Quantity"] > 0]
data = data[data["Price"] > 0]

In [20]:
# Creating a new column 'Total_Prize' to calculate total price for each product
data["Total_Prize"] = data["Quantity"] * data["Price"]

In [21]:
data.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Prize
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.950,13085.000,United Kingdom,83.400
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.750,13085.000,United Kingdom,81.000
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.100,13085.000,United Kingdom,100.800
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.250,13085.000,United Kingdom,30.000


In [22]:
Df_info(data)

########### Head ###########
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  Total_Prize  
0 2009-12-01 07:45:00  6.950    13085.000  United Kingdom       83.400  
1 2009-12-01 07:45:00  6.750    13085.000  United Kingdom       81.000  
2 2009-12-01 07:45:00  6.750    13085.000  United Kingdom       81.000  
3 2009-12-01 07:45:00  2.100    13085.000  United Kingdom      100.800  
4 2009-12-01 07:45:00  1.250    13085.000  United Kingdom       30.000  
########### Tail ###########
        Invoice StockCode                      Description  Quantity  \
1067366  58

In [23]:
Df_info2(data)

The number of customer in the dataset: 5878
-------------------------------------------------------
The number of invoice in the dataset: 36969
-------------------------------------------------------
The sum of quantity in the dataset: 10706167
-------------------------------------------------------
The sum of price in the dataset: 17743429
-------------------------------------------------------
Our dataset, sort by highest salary
       Invoice StockCode Description  Quantity         InvoiceDate     Price  \
135013  502263         M      Manual         1 2010-03-23 15:22:00 10953.500   
358639  524159         M      Manual         1 2010-09-27 16:12:00 10468.800   
74356   496115         M      Manual         1 2010-01-29 11:04:00  8985.600   
698843  551697      POST     POSTAGE         1 2011-05-03 13:46:00  8142.750   
129987  501768         M      Manual         1 2010-03-19 11:45:00  6958.170   

        Customer ID         Country  Total_Prize  
135013    12918.000  United Kingd

In [ ]:
#we are successfully read, cleaned and ready for further analysis

In [25]:
# we will save the cleaned data to a new CSV file for future use
df.to_csv('cleaned_data.csv', index=False)
print("✅ Temizlenmiş veri 'cleaned_data.csv' successfully saved.")

✅ Temizlenmiş veri 'cleaned_data.csv' successfully saved.
